# GCS Access

In [ ]:
import tempfile
import uuid
from pathlib import Path

from collab_env.data.gcs_utils import GCSClient

%load_ext autoreload
%autoreload 2

## Example 2: create and manupulate a remote bucket

In [2]:
BUCKET_NAME = f"test-bucket-collab-data_{uuid.uuid4()}"

# Connect to GCS and create a the bucket
# use default credentials for collab-data project
gcs_client = GCSClient()

gcs_client.create_bucket(BUCKET_NAME)

print(f"globbing the bucket {BUCKET_NAME}:\n\n{gcs_client.glob(f'{BUCKET_NAME}/*')}")

2025-07-23 18:46:51.274 | INFO     | collab_env.data.gcs_utils:__init__:34 - Using credentials from /Users/dima/git/collab-environment/config-local/collab-data-463313-c340ad86b28e.json
2025-07-23 18:46:51.275 | INFO     | collab_env.data.gcs_utils:__init__:40 - Using project collab-data-463313
2025-07-23 18:46:52.379 | INFO     | collab_env.data.gcs_utils:create_bucket:66 - Created bucket test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402.


globbing the bucket test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402:

[]


In [3]:
all_buckets = gcs_client.list_buckets()

if BUCKET_NAME not in all_buckets:
    print(f"Bucket {BUCKET_NAME} does not exist, creating it...")
    gcs_client.create_bucket(BUCKET_NAME)
else:
    print(f"Bucket {BUCKET_NAME} already exists")

print(f"Available buckets: {gcs_client.list_buckets()}")

Bucket test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402 already exists
Available buckets: ['fieldwork_curated', 'fieldwork_processed', 'test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402']


### Create a folder for the bucket

In [4]:
# create a folder in the bucket
folder_name = f"test_folder_{uuid.uuid4()}"

gcs_client.create_folder(f"{BUCKET_NAME}/{folder_name}")

glob_str = f"{BUCKET_NAME}/{folder_name}/*"
print(f"globbing the folder {folder_name}:\n\n{gcs_client.glob(glob_str)}")

2025-07-23 18:46:53.340 | INFO     | collab_env.data.gcs_utils:create_folder:119 - Created folder marker at gs://test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.folder_marker


globbing the folder test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04:

['test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.folder_marker']


### Adding files to bucket

Create a file locally and upload to GCS bucket

In [5]:
# upload a file to the folder
# create a temp file
with tempfile.NamedTemporaryFile(delete=True) as temp_file:
    temp_file.write(b"test file content")
    temp_file.flush()

    gcs_client.upload_file(temp_file.name, f"{BUCKET_NAME}/{folder_name}/")

print(f"globbing the folder {folder_name}:\n\n{gcs_client.glob(glob_str)}")

2025-07-23 18:46:53.390 | INFO     | collab_env.data.gcs_utils:upload_file:179 - Uploading file /var/folders/2j/jdg5vjdx4x3_5tmlzl7nzygh0000gn/T/tmpu_ovefeb to test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.
2025-07-23 18:46:53.560 | INFO     | collab_env.data.gcs_utils:upload_file:181 - Uploaded file /var/folders/2j/jdg5vjdx4x3_5tmlzl7nzygh0000gn/T/tmpu_ovefeb to test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.


globbing the folder test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04:

['test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.folder_marker', 'test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/tmpu_ovefeb']


### Access files from GCS bucket

In [6]:
# list file contents
with gcs_client.gcs.open(
    f"{BUCKET_NAME}/{folder_name}/{Path(temp_file.name).stem}", "r"
) as f:
    content = f.read()
    print(f"File content:\n{content}")

File content:
test file content


### Remove file from bucket

In [7]:
# remove the file
gcs_client.delete_path(f"{BUCKET_NAME}/{folder_name}/{Path(temp_file.name).stem}")

print(f"globbing the folder {folder_name}:\n\n{gcs_client.glob(glob_str)}")

2025-07-23 18:46:53.935 | INFO     | collab_env.data.gcs_utils:delete_path:192 - Deleted path test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/tmpu_ovefeb.


globbing the folder test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04:

['test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.folder_marker']


### Remove the created test folder

In order for a folder to be removed it must be empty

In [8]:
# remove the folder
gcs_client.remove_folder(f"{BUCKET_NAME}/{folder_name}/")
bucket_glob_str = f"{BUCKET_NAME}/*"
print(f"globbing the bucket {BUCKET_NAME}:\n\n{gcs_client.glob(bucket_glob_str)}")

2025-07-23 18:46:54.379 | INFO     | collab_env.data.gcs_utils:remove_folder:155 - Removed folder marker at gs://test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/.folder_marker
2025-07-23 18:46:54.503 | INFO     | collab_env.data.gcs_utils:remove_folder:160 - Removed folder test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402/test_folder_be9cf0a2-4dae-4044-90fd-8cfe56889b04/


globbing the bucket test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402:

[]


### Delete the test bucket

In order for a bucket to be removed, it must be empty (no folders)

In [9]:
# remove the bucket
gcs_client.delete_bucket(BUCKET_NAME)

all_buckets = gcs_client.list_buckets()
print(f"Available buckets: {all_buckets}")

2025-07-23 18:46:55.079 | INFO     | collab_env.data.gcs_utils:delete_bucket:86 - Deleted bucket test-bucket-collab-data_b132886a-f1ec-4b6d-af82-4727cd9a4402.


Available buckets: ['fieldwork_curated', 'fieldwork_processed']
